instalaciones e imports previos

In [ ]:
pip install tfimm

In [ ]:
pip install timm

In [ ]:
import tfimm
import timm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
model_list = tfimm.list_models(pretrained="timm")
str_match = [s for s in model_list if "vit" in s]
str_match

In [ ]:
vit_model = tfimm.create_model("vit_base_patch32_224", pretrained="timm")
vit_model.summary()

image size de nuestro ViT

In [ ]:
image_size = 224

input shape de CIFAR-10

In [ ]:
input_shape = (32, 32, 3)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
preprocess = tfimm.create_preprocessing("vit_base_patch32_224", dtype="float32")
x_train, x_test = preprocess(x_train), preprocess(x_test)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        )
    ],
    name="data_augmentation",
)

añadimos las capas de entrada y salida correspondiente a nuestro problema (CIFAR-10)

In [ ]:
inputs = layers.Input(shape=input_shape)
augmented = data_augmentation(inputs)
features = vit_model(augmented)
logits = keras.layers.Dense(10)(features)
model = keras.Model(inputs=inputs, outputs=logits)
model.summary()

In [ ]:
"""
model = keras.models.Sequential()
model.add(layers.Input(shape=input_shape))
model.add(data_augmentation)
for layer in vit_model.layers[0:-1]:
    model.add(layer)
model.add(keras.layers.Dense(10))
model.summary()

"""

hacemos las capas no entrenables

In [ ]:
#for layer in model.layers:
#    layer.trainable = False
#model.summary()

parámetros para compilar, entrenar y evaluar el modelo

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 30

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x= x_train,
        y= y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history

history = run_experiment(model)